# NCEI
- 월평균 활용
- https://apihub.kma.go.kr/
    - 세계기상 > NCEI 관측.통계 > 3. 전세계 지상관측(월통계) : 1763년~2022년
    - ※ NCEI(National Centers for Environmental Information, 국립환경정보센터)는 미국 NOAA의 소속된 기관으로 전세계 기상기후데이터를 수집·제공합니다.
    - ※ 모든 자료가 2022년 12월 까지 존재합니다.
## 참고 자료
- document 파일
    - 자료 소개서
        - GSOM_readme.txt
    - 자료 설명서
        - GSOM_documentation.pdf
    - 연/월 통합 상세 설명서
        - GSOM_GSOY_Description_Document_v1.0.2_20200219.pdf
- 지점정보
    - 링크 내용은 동일함
    - https://www.ncei.noaa.gov/pub/data/noaa/isd-history.txt(내용확인)
    - https://www.ncei.noaa.gov/pub/data/noaa/isd-history.csv(파일다운로드)
- 국가 코드
    - Country list
        - 국가명과 국가 2글자 표현 짝 정리된 것
        - 후에 요청인자 중 stn 구성요소인 지점코드값의 국가 부분을 해석하는데에 사용
- 파일 리스트
    - File list
        - 파일목록
        - 파일명과 파일 크기가 나열됨
        - 후술될 요청인자 중 stn의 구성요소인 지점코드 값으로 활용할 수 있음
            - 국가 2글자, 망구분 1글자, 지점번호 9글자를 가져올 소스
## 참고 사이트
- 바이너리 변환: https://m.blog.naver.com/errorsoft666/222132918275
- 

## 활용 library

In [1]:
import pandas as pd
import numpy as np
import io
import requests
import urllib
from urllib.request import urlopen

In [2]:
# 개인 고유 인증키
# 각자 버전 활용해야 함
AUTH = "secrets Leak alerts로 인해 마스킹처리, 코드 run 원할시 기상청 API 허브 가입 후 활용해 주세요."

In [3]:
# 파일 저장하는 함수
def download_file(file_url, save_path):
    with open(save_path, 'wb') as f: # 저장할 파일을 바이너리 쓰기 모드로 열기
        response = requests.get(file_url) # 파일 URL에 GET 요청 보내기
        f.write(response.content) # 응답의 내용을 파일에 쓰기

## Country list
- country-list.txt 파일을 불러와 DF로 정리
- csv로 저장하기

In [4]:
dfCountry = pd.read_table("rawdata_climate/country-list.txt", sep = '    ', encoding = 'euc-kr', engine = 'python')
dfCountry

,FIPS ID,COUNTRY NAME
AA,NaN,ARUBA
AC,NaN,ANTIGUA AND BARBUDA
AF,NaN,AFGHANISTAN
AG,NaN,ALGERIA
AI,NaN,ASCENSION ISLAND
...,...,...
YY,NaN,"ST. MARTEEN, ST. EUSTATIUS, AND SABA"
ZA,NaN,ZAMBIA
ZI,NaN,ZIMBABWE
ZM,NaN,SAMOA


In [5]:
dfCountry.reset_index(inplace = True)
dfCountry['FIPS ID'] = dfCountry['index']
dfCountry.drop('index', inplace = True, axis = 1)

In [6]:
dfCountry

,FIPS ID,COUNTRY NAME
0,AA,ARUBA
1,AC,ANTIGUA AND BARBUDA
2,AF,AFGHANISTAN
3,AG,ALGERIA
4,AI,ASCENSION ISLAND
...,...,...
288,YY,"ST. MARTEEN, ST. EUSTATIUS, AND SABA"
289,ZA,ZAMBIA
290,ZI,ZIMBABWE
291,ZM,SAMOA


In [7]:
dfCountry.to_csv('data_climate/country_list.csv', index = False)

## 전세계 지상관측(월통계) : 1763년~2022년
- filelist 내용
--------------------------------------------------------------------------------------------------
미국 NCEI 전세계 지상관측 월통계의 연도별 파일목록  
예) http://api.kma.go.kr/url/ncei_gsom_list.php
--------------------------------------------------------------------------------------------------
1. FILE : file name (지점코드.csv)
2. SIZE : file size (bytes)
--------------------------------------------------------------------------------------------------
FILE,SIZE

In [8]:
# URL과 저장 경로 변수를 지정합니다.
url = "https://apihub.kma.go.kr/api/typ01/url/ncei_gsom_list.php?authKey=" + AUTH
save_file_path = 'rawdata_climate/filelist.txt'

# 파일 다운로드 함수를 호출합니다.
download_file(url, save_file_path)

In [9]:
dfFileList = pd.read_table("rawdata_climate/filelist.txt", sep = ',', header = 7,
                           encoding = 'euc-kr', engine = 'python', )
dfFileList

,# FILE,SIZE
0,gsom_ACW00011604.csv,4036
1,gsom_ACW00011647.csv,34313
2,gsom_AE000041196.csv,130742
3,gsom_AEM00041194.csv,100242
4,gsom_AEM00041217.csv,85259
...,...,...
122231,gsom_ZI000067965.csv,114617
122232,gsom_ZI000067969.csv,116596
122233,gsom_ZI000067975.csv,200883
122234,gsom_ZI000067977.csv,84153


In [10]:
dfFileList.columns = ["FILE", "SIZE"]
dfFileList

,FILE,SIZE
0,gsom_ACW00011604.csv,4036
1,gsom_ACW00011647.csv,34313
2,gsom_AE000041196.csv,130742
3,gsom_AEM00041194.csv,100242
4,gsom_AEM00041217.csv,85259
...,...,...
122231,gsom_ZI000067965.csv,114617
122232,gsom_ZI000067969.csv,116596
122233,gsom_ZI000067975.csv,200883
122234,gsom_ZI000067977.csv,84153


In [11]:
dfFileList.to_csv('data_climate/file_list.csv', index = False)

# 여기서부터 파일 누적하여 만들어야 함
- 윗부분은 이미 다 만들어서 여러 번 할 필요 X

## 전세계 지상관측(월통계) 호출 방법
- 호출URL정보 예시
    - 여러 지점
        - https://apihub.kma.go.kr/api/typ01/url/ncei_gsom_data.php?tm=202208&stns=KSM00047108,KSM00047159&authKey=AUTH
    - 시간 구간
        - https://apihub.kma.go.kr/api/typ01/url/ncei_gsom_data.php?tm1=202201&tm2=202208&stns=KSM00047108,KSM00047159&authKey=AUTH
    - 임의 영역내
        - https://apihub.kma.go.kr/api/typ01/url/ncei_gsom_data.php?tm=202208&lon1=125&lon2=130&lat1=32&lat2=38&authKey=AUTH
    - 1개 지점의 전체 자료
        - https://apihub.kma.go.kr/api/typ01/url/ncei_gsom_file.php?stn=KSM00047108&authKey=AUTH
### 요청 인자
- tm
    - 년월일시분(UTC)
    - 특정시간대만 추출할 때
- tm1
    - 년월일시분(UTC)
    - tm1~tm2 기간의 자료를 추출
- tm2
    - 년월일시분(UTC)
    - tm1~tm2 기간의 자료를 추출
- stns
    - 지점들
    - 여러지점코드를 ,로 구분하면 같이 처리함
    - *) 지점코드 : 국가(2글자)+망구분(1글자)+지점번호(9글자)
- lon1
    - 영역
    - 경도 [lon1~lon2] 내 자료들만 추출 (앞에 stns가 정의되어 있으면, stns을 우선함)
- lon2
    - 영역
    - 경도 [lon1~lon2] 내 자료들만 추출 (앞에 stns가 정의되어 있으면, stns을 우선함)
- lat1
    - 영역
    - 위도 [lat1~lat2] 내 자료들만 추출 (앞에 stns가 정의되어 있으면, stns을 우선함)
- lat2
    - 영역
    - 위도 [lat1~lat2] 내 자료들만 추출 (앞에 stns가 정의되어 있으면, stns을 우선함)
- authKey
    - 인증키
    - 발급된 API 인증키

- 모든 나라의 데이터 기간을 통일하기 위해 임의로 200001부터 202212까지로 정해서 불러와본다.
- stn 코드값은 위의 filelist의 FILE 컬럼 내부 값을 조정하여 활용한다.

In [12]:
stnList = []
for idx, row in dfFileList.iterrows():
    tmp = row[0][5:-4]
    stnList.append(tmp)
print(len(stnList))
print(stnList[:5])

122236
['ACW00011604', 'ACW00011647', 'AE000041196', 'AEM00041194', 'AEM00041217']


### 1개 지점의 전체 자료를 txt로 저장한 후 csv로 변환해 저장
- 굳이 txt로 저장한 후 csv로 저장할 필요 있나 싶어 csv로 바로 저장했는데 문제가 없어서 일단 두 버전 둠

In [14]:
# txt > csv
for STN in stnList[:10]:

    # URL과 저장 경로 변수를 지정합니다.
    domain = "https://apihub.kma.go.kr/api/typ01/url/ncei_gsom_file.php?"

    stns = "stn=" + STN
    authKey = "&authKey=" + AUTH

    url = domain + stns + authKey

    save_file_path = 'rawdata_climate/' + str(STN) + '.txt'

    # 파일 다운로드 함수를 호출합니다.
    download_file(url, save_file_path)
    
varList = []
for STN in stnList[:10]:
    varNM = 'df_{}'.format(STN)
    
    globals()[varNM] = pd.read_table('rawdata_climate/' + str(STN) + '.txt', sep = ',',
                                      encoding = 'euc-kr', engine = 'python')
    varList.append(varNM)
    
    globals()[varNM].to_csv('data_climate/' + str(STN) + '.csv', index = False)

# 변수명 리스트
print(varList)

# 변수명 리스트에서 str 값을 변수로 활용하는 방법
globals()[varList[0]]

In [24]:
# csv로 바로 저장

startNum = 0
endNum = 100

for i, STN in enumerate(stnList[startNum:endNum]):

    # URL과 저장 경로 변수를 지정합니다.
    domain = "https://apihub.kma.go.kr/api/typ01/url/ncei_gsom_file.php?"

    stns = "stn=" + STN
    authKey = "&authKey=" + AUTH

    url = domain + stns + authKey

    save_file_path = 'data_climate/' + str(STN) + '.csv'

    # 파일 다운로드 함수를 호출합니다.
    download_file(url, save_file_path)
    
    # 확인용 호출
    print(STN)
    
    if i % 10 == 0:
        print('-----------------------------------')
        print(i + '번째')
    
varList = []
for STN in stnList[startNum:endNum]:
    varNM = 'df_{}'.format(STN)
    
    globals()[varNM] = pd.read_csv('data_climate/' + str(STN) + '.csv', sep = ',',
                                      encoding = 'euc-kr', engine = 'python')
    varList.append(varNM)
    
# 변수명 리스트
print(varList)

# 변수명 리스트에서 str 값을 변수로 활용하는 방법
# 랜덤으로 확인하기
tmp = int((startNum + endNum) / 2)
globals()[varList[tmp]]

ACW00011604
ACW00011647
AE000041196
AEM00041194
AEM00041217
AEM00041218
AF000040930
AFM00040938
AFM00040948
AFM00040990
AG000060390
AG000060590
AG000060611
AG000060680
AGE00135039
AGE00147704
AGE00147705
AGE00147706
AGE00147707
AGE00147708
AGE00147709
AGE00147710
AGE00147711
AGE00147712
AGE00147713
AGE00147714
AGE00147715
AGE00147716
AGE00147717
AGE00147718
AGE00147719
AGE00147720
AGE00147780
AGE00147794
AGM00060351
AGM00060353
AGM00060355
AGM00060360
AGM00060367
AGM00060369
AGM00060387
AGM00060402
AGM00060403
AGM00060405
AGM00060410
AGM00060415
AGM00060417
AGM00060419
AGM00060421
AGM00060423
AGM00060425
AGM00060430
AGM00060437
AGM00060444
AGM00060445
AGM00060452
AGM00060457
AGM00060461
AGM00060467
AGM00060468
AGM00060471
AGM00060475
AGM00060476
AGM00060490
AGM00060506
AGM00060507
AGM00060511
AGM00060514
AGM00060515
AGM00060518
AGM00060520
AGM00060522
AGM00060531
AGM00060535
AGM00060536
AGM00060540
AGM00060549
AGM00060550
AGM00060555
AGM00060557
AGM00060559
AGM00060560
AGM00060563
AGM0

UnicodeDecodeError: 'euc_kr' codec can't decode byte 0x80 in position 79: illegal multibyte sequence

# 오류 발생 가능성 때문에 txt로 시작할까 고민중!!!
# 근데 csv 파일 전체 다운하는것을 발견했다!!!!!!!!
# 그래도 유럽 국가만 고르기 위해 기준 파일은 필요할 듯!!!